In [1]:
import pandas as pd
import os
import yt_dlp
import yaml
import argparse
import whisperx
import torch
import json
import time
import shutil

from pathlib import Path
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
from pyannote.audio import Audio
from pyannote.core import Segment
from scipy.spatial.distance import cdist

torchvision is not available - cannot save figures


In [2]:
hf_token = "<hf-token>" # https://huggingface.co/settings/tokens
cdist_threshold = 0.5 # see below

In [3]:
reference = {
    "MILEI PRESIDENTE": "milei",
    "El Peluca Milei": "milei",
    "Javier Milei": "milei",
    "Sergio Massa": "massa",
    "Patricia Bullrich": "bullrich",
}

In [4]:
data_dir = '../data/youtube_data'
ref_audio_dir = '../data/reference_audio'
temp_dir = '../data/temp_audio'
output_dir = '../output'

In [5]:
target_audio = f"{temp_dir}/audio.wav"
input_data = f'{data_dir}/data.csv'

In [6]:
# env initialization
device = "cuda"
batch_size = 32
compute_type = "float16"

In [7]:
data = pd.read_csv(input_data)

In [8]:
data.head()

,channel_id,channel_name,channel_description,channel_uploader_url,channel_title,id,url,title,description,duration,view_count
0,UCwKZnJ48oXRCCGCQbL1BPSA,Javier Milei,Canal Oficial de Javier Milei\nVLLC!\n\n,https://www.youtube.com/@JavierMileiOK,Javier Milei - Videos,zAjBSfFGpKk,https://www.youtube.com/watch?v=zAjBSfFGpKk,MILEI DESTROZA A LUIS MAJUL,Canal Oficial de Javier Milei\n\nRedes:\nInsta...,1703.0,27154
1,UCwKZnJ48oXRCCGCQbL1BPSA,Javier Milei,Canal Oficial de Javier Milei\nVLLC!\n\n,https://www.youtube.com/@JavierMileiOK,Javier Milei - Videos,yAuIjst9ZIU,https://www.youtube.com/watch?v=yAuIjst9ZIU,JAVIER MILEI CON BABY ETCHECOPAR,Redes:\nInstagram: https://www.instagram.com/j...,2671.0,6738
2,UCwKZnJ48oXRCCGCQbL1BPSA,Javier Milei,Canal Oficial de Javier Milei\nVLLC!\n\n,https://www.youtube.com/@JavierMileiOK,Javier Milei - Videos,3pFGfppjiQs,https://www.youtube.com/watch?v=3pFGfppjiQs,JAVIER MILEI RESPONDE TODO EN TN,Redes:\n\nInstagram: https://www.instagram.com...,2495.0,1981
3,UCwKZnJ48oXRCCGCQbL1BPSA,Javier Milei,Canal Oficial de Javier Milei\nVLLC!\n\n,https://www.youtube.com/@JavierMileiOK,Javier Milei - Videos,dsq0YvdQmIE,https://www.youtube.com/watch?v=dsq0YvdQmIE,JAVIER MILEI CON JONATAN VIALE EN LN+,Canal Oficial de Javier Milei\n\nRedes:\nInsta...,3306.0,1530
4,UCwKZnJ48oXRCCGCQbL1BPSA,Javier Milei,Canal Oficial de Javier Milei\nVLLC!\n\n,https://www.youtube.com/@JavierMileiOK,Javier Milei - Videos,rTR8UdBK8n4,https://www.youtube.com/watch?v=rTR8UdBK8n4,"Javier Milei: ""Ningún rival directo me llamó p...",Canal Oficial de Javier Milei\n\nRedes:\nInsta...,2435.0,3252


In [9]:
def download_audio(URL, output_dir, audio_fname:str = 'audio'):
    """
    Download the audio from a YouTube video as a WAV file.
    Args:
        URL (str): The URL of the YouTube video.
        output_dir (str): The directory to save the output file in.
        audio_filename (str): The filename to save the output file as.
    """
    shutil.rmtree(output_dir)
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    ydl_opts = {
        'outtmpl': os.path.join(output_dir, audio_fname),
        'format': 'm4a/bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
        }]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download(URL)

In [10]:
download_audio(data.iloc[0].url, temp_dir)

[youtube] Extracting URL: https://www.youtube.com/watch?v=zAjBSfFGpKk
[youtube] zAjBSfFGpKk: Downloading webpage


[youtube] zAjBSfFGpKk: Downloading ios player API JSON
[youtube] zAjBSfFGpKk: Downloading android player API JSON
[youtube] zAjBSfFGpKk: Downloading iframe API JS
[youtube] zAjBSfFGpKk: Downloading player 16f9263d
[youtube] zAjBSfFGpKk: Downloading web player API JSON
[youtube] zAjBSfFGpKk: Downloading m3u8 information


[youtube] zAjBSfFGpKk: Downloading initial data API JSON
[info] zAjBSfFGpKk: Downloading 1 format(s): 140
[download] Destination: ../data/temp_audio/audio
[download] 100% of   26.28MiB in 00:00:01 at 25.82MiB/s  
[FixupM4a] Correcting container of "../data/temp_audio/audio"
[ExtractAudio] Destination: ../data/temp_audio/audio.wav
Deleting original file ../data/temp_audio/audio (pass -k to keep)


In [11]:
# transcription model
model = whisperx.load_model("large-v2", device, compute_type=compute_type)

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.7. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../.cache/torch/whisperx-vad-segmentation.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1+cu117. Bad things might happen unless you revert torch to 1.x.


In [12]:
# transcribe and perform speaker diarization
audio = whisperx.load_audio(target_audio)
result = model.transcribe(audio, batch_size=batch_size, language='es')

In [13]:
# align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

In [14]:
# assign speaker labels
diarize_model = whisperx.DiarizationPipeline(use_auth_token=hf_token, device=device)
diarize_segments = diarize_model(audio)
result = whisperx.assign_word_speakers(diarize_segments, result)

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.7. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1+cu117. Bad things might happen unless you revert torch to 1.x.


In [15]:
# load embeddings model
embeddings_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=device)

In [16]:
# start audio pipeline
audio_pipe = Audio(sample_rate=16000, mono="downmix")
reference_wav_fname = f'{ref_audio_dir}/audio_{reference[data.iloc[0].channel_name]}.wav'

In [17]:
# extract embeddings for a speaker speaking between t=0 and t=10s
segment_reference = Segment(1., 10.)
waveform_reference, sample_rate = audio_pipe.crop(reference_wav_fname, segment_reference)
embedding_reference = embeddings_model(waveform_reference[None])

In [18]:
stime = time.time()

for segment in result['segments']:
    # extract embedding for a speaker speaking between t=Xs and t=Ys
    speaker_target = Segment(segment['start'], segment['end'])
    waveform_target, sample_rate = audio_pipe.crop(target_audio, speaker_target)
    embedding_target = embeddings_model(waveform_target[None])

    # compare embeddings using "cosine" distance
    distance = cdist(embedding_reference, embedding_target, metric="cosine")
    segment['cosine_dist'] = distance[0][0]

    # save back the info to the dict
    segment['is_candidate'] = True if distance[0][0] <= cdist_threshold else False

print(f'total time: {time.time()-stime:.2f} seconds')

total time: 8.39 seconds


In [19]:
def save_dict_to_disk(data_dict, data, reference, filename:str = None):
    """
    Save a dictionary to disk as a JSON file.
    Args:
        data_dict (dict): The dictionary to save.
        data (DataFrame): A dataframe containing a column 'channel_name'.
        reference (dict): A dictionary mapping channel names to some values.
        filename (str): The filename to save the output file as (without extension).
    """
    output_dir = f"../output/{reference[data.channel_name]}"
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    filepath = os.path.join(output_dir, f"{filename}.json")
    
    combined_dict = data_dict | data.to_dict()
    with open(filepath, 'w') as f:
        json.dump(combined_dict, f)
    
    print(f"dict saved to {filepath}")

In [20]:
save_dict_to_disk(result, data.iloc[0], reference, filename = data.iloc[0].id)

dict saved to ../output/milei/zAjBSfFGpKk.json
